# 1. Enviornment setup

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import csv
# import torch
import numba
from numba import jit, cuda
# import torchvision.models as models
# from torch import nn, optim
# import timm 

# multi-threading
import concurrent.futures

# max depth of the image
max_depth = 100.0

# Use half of cpu for multi-threading tasks
# Consider use gpu if CUDA is available
num_workers = os.cpu_count() / 2
print("Number of CPU will be used:", num_workers)

weight_data = 'Data/final_mapping_original.csv'

# cuda_available = torch.cuda.is_available()
# print("CUDA available:", cuda_available)
# if cuda_available:
#     print("CUDA version:", torch.version.cuda)

OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\Niko\anaconda3\envs\PigPixel\Lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.

# 2. Pre-processing

### 2.1 Filtering out background

Filter out all background and leave a pure pig depth image to process

In [3]:
def img_filter(file_addr):
    
    # Load image
    image = cv2.imread(file_addr)

    # Adjust GMM parameters
    backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False)
    fgMask = backSub.apply(image)

    # Refine the foreground mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.erode(fgMask, kernel, iterations=2)
    fgMask = cv2.dilate(fgMask, kernel, iterations=2)

    # Convert image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define color range for the pig color
    lower_color = np.array([110, 255, 254])
    upper_color = np.array([255, 255, 255])
    color_mask = cv2.inRange(hsv, lower_color, upper_color)

    # Combine color mask with foreground mask
    fgMask = cv2.bitwise_and(fgMask, color_mask)

    # Find contours from the mask
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the pig and create a mask for it
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        refined_mask = np.zeros_like(fgMask)
        cv2.fillPoly(refined_mask, [largest_contour], 255)
        fgMask = refined_mask

    # Extract the foreground using the refined mask
    foreground = cv2.bitwise_and(image, image, mask=fgMask)
    
    return foreground

Test the filter result

In [ ]:
# eg_img = 'Data/Week2/20211002/20211002_3342/_Depth_4780.png' # Good example
# eg_img = 'Data/Week2/20211002/20211002_3342/_Depth_3223.png' # Good example
# eg_img = 'Data/Week2/20211003/20211003_3342a/_Depth_4489.png' # not that bad example
eg_img = 'Data/Week1/20210922/20210922_3342/_Depth_5395.png' # Bad example
foreground = img_filter(eg_img)

# image = cv2.imread(eg_img)
# print("Total number of pixels after filtering:", image.shape[0] * image.shape[1])
# cv2.imshow('Foreground', foreground)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### 2.2 Count pixels

Count the pixels in the image

In [4]:
def get_pixel_map(img):

    # Convert the foreground to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten the image
    pixels = img.reshape(-1, img.shape[-1])

    # Count unique colors
    unique_colors, counts = np.unique(pixels[pixels.sum(axis=1) != 0], axis=0, return_counts=True)

    # Create a color map
    color_map = {tuple(color): count for color, count in zip(unique_colors, counts)}
    
    return color_map

def get_total_heights(img):

    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Normalize the depth values (assuming 8-bit image and max depth of 10 meters)
    # You will need to adjust the max_depth to match the actual range of your sensor
    
    normalized_depth_image = (gray_image / 255.0) * max_depth
    
    total_height = np.sum(normalized_depth_image)
    
    # cv2.imshow('Image', normalized_depth_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return total_height

def get_mean_heights(img):

    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Normalize the depth values (assuming 8-bit image and max depth of 10 meters)
    # You will need to adjust the max_depth to match the actual range of your sensor
    
    normalized_depth_image = (gray_image / 255.0) * max_depth
    
    mean_height = np.sum(normalized_depth_image) / np.count_nonzero(normalized_depth_image)
    
    # cv2.imshow('Image', normalized_depth_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return mean_height
    

Test counting pixels

In [8]:
eg_img = 'Data/Week2/20211002/20211002_3342/_Depth_4780.png'
foreground = img_filter(eg_img)
# total_heights = get_total_heights(foreground)
mean_heights = get_mean_heights(foreground)
# color_map = get_pixel_map(foreground)
# print("Total number of pixels in map:", sum(color_map.values()))
# print("Color Map:", color_map)
print("Total height: ", mean_heights)

Total height:  21.164121994912374


### 2.3 Mapping weight w/ pixel map
Read weight w/ the pixel map  
Use weight, image_path, color_map to access the data

In [5]:

def img_to_data(row):
    image_path = 'Data/' + row[0]
    base_name, _ = os.path.splitext(image_path)
    image_path = f'{base_name}.png' # Change from jpg to png
    # print(image_path)
    
    # if using whole image files, use this one
    if not os.path.exists(image_path): 
        # print(f"Image not found: {image_path}")
        return
    
    # Get the foreground by calling img_filter
    foreground = img_filter(image_path)
    
    # Get the pixel map by calling get_pixel_map
    # color_map = get_pixel_map(foreground)
    
    # total_height = get_total_heights(foreground)
    mean_height = get_mean_heights(foreground)
    
    # print(f"Height: {mean_heights}")
    
    return {
                'weight': row[3],
                'image_path': image_path,
                # 'color_map': color_map
                'mean_height': mean_height
            }
            
def get_data():

    with open(weight_data, mode='r', newline='') as infile:
        reader = csv.reader(infile)

        # Create a new list to hold the combined data
        combined_data = []
        headers = next(reader)
        
        # multi-threading data process
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Start a set of tasks and mark each future with its URL
            future_to_task = {executor.submit(img_to_data, arg): arg for arg in reader}
            for future in concurrent.futures.as_completed(future_to_task):
                arg = future_to_task[future]
                try:
                    data = future.result()
                    if data is not None:
                        combined_data.append(data)
                except Exception as exc:
                    print(f'{arg} generated an exception: {exc}')
                # else:
                #     print(f'{arg} page is {len(data)} bytes')
                
    # print(combined_data)
    # Convert the combined data to a DataFrame
    combined_df = pd.DataFrame(combined_data)

    # Display the combined DataFrame
    print(combined_df)
    return combined_df

### 2.4 Gather all data and export the result

In [7]:
data = get_data()

csv_file_path = 'Data/data.csv'
data.to_csv(csv_file_path, index=False)

[{'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2702.png', 'mean_height': 21.67668943600382}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2086.png', 'mean_height': 21.67823647575242}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2378.png', 'mean_height': 20.678843708578306}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2180.png', 'mean_height': 21.711833532946134}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_1897.png', 'mean_height': 21.680096298465514}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2986.png', 'mean_height': 21.765984742156927}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2692.png', 'mean_height': 21.71257435977559}, {'weight': '47.6', 'image_path': 'Data/Week9/20211112/20211112_3342/_Depth_2317.png', 'mean_height': 21.557036607746543}, {'weight': '47.6', 'image_